In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from pymongo import MongoClient

In [2]:
def salary(lst):
    if lst[0] == "до":
        return {"min": None, "max": int(lst[1] + lst[2])}
    if lst[0] == "от":
        return {"min": int(lst[1] + lst[2]), "max": None}
    if lst[0].isnumeric():
        if len(lst) > 3:
            return {"min": int(lst[0] + lst[1]), "max": int(lst[3] + lst[4])}
        else:
            return {"min": int(lst[0] + lst[1]), "max": int(lst[0] + lst[1])}
    return {"min": None, "max": None}

In [3]:
def getPositions(dom):
    positions = dom.find_all("div", {"class": "f-test-search-result-item"})
    positions_list = []

    for position in positions:
        position_features = []

        position_link = position.find("a")
        position_salary = position.find("span", {"class": "_2Wp8I _3a-0Y _3DjcL _1tCB5 _3fXVo"})

        # Борьба с рекламными блоками
        if position_link == None or position_salary == None:
            continue

        position_features.append(position_link.getText())
        position_features.append(salary(position_salary.getText().split())["min"])
        position_features.append(salary(position_salary.getText().split())["max"])
        position_features.append(url_base+position_link["href"])
        position_features.append(url_base)

        positions_list.append(position_features)
    
    return positions_list

In [4]:
url_base = "https://www.superjob.ru"
#url = "/vacancy/search/?keywords=Data%20Science&geo%5Bt%5D%5B0%5D=4"
url = "/vacancy/search/?keywords=Java&geo%5Bt%5D%5B0%5D=4"
#url = "/vacancy/search/?keywords=Python&geo%5Bt%5D%5B0%5D=4"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"
}

In [5]:
response = requests.get(url_base+url, headers=headers)
dom = BeautifulSoup(response.text, "html.parser")

In [6]:
pages = dom.find("div", {"class": "_2G0xv L1p51 bwVVU VsleO e495U _2_kCs _3l5cC"})
positions_list = getPositions(dom)

if pages != None:
    pages = pages.find_all("a")
    for i in range(2, len(pages)):
        next_page = f"&page={i}"
        response = requests.get(url_base+url+next_page, headers=headers)
        dom = BeautifulSoup(response.text, "html.parser")
        positions_list += getPositions(dom)

In [7]:
pd.DataFrame(positions_list, columns=["Название вакансии", "Мин. зарплата", "Макс. зарплата", "Ссылка на вакансию", "Сайт"])

,Название вакансии,Мин. зарплата,Макс. зарплата,Ссылка на вакансию,Сайт
0,Java developer,NaN,200000.0,https://www.superjob.ru/vakansii/java-develope...,https://www.superjob.ru
1,Разработчик Java / Программист Java (Junior / ...,91000.0,NaN,https://www.superjob.ru/vakansii/razrabotchik-...,https://www.superjob.ru
2,Ведущий разработчик Java,NaN,NaN,https://www.superjob.ru/vakansii/veduschij-raz...,https://www.superjob.ru
3,Java-разработчик (Трайб Розничное взыскание),NaN,NaN,https://www.superjob.ru/vakansii/java-razrabot...,https://www.superjob.ru
4,Java-разработчик,NaN,NaN,https://www.superjob.ru/vakansii/java-razrabot...,https://www.superjob.ru
5,Разработчик Java,250000.0,NaN,https://www.superjob.ru/vakansii/razrabotchik-...,https://www.superjob.ru
6,Java-разработчик (инженер),NaN,NaN,https://www.superjob.ru/vakansii/java-razrabot...,https://www.superjob.ru
7,Java-developer,NaN,300000.0,https://www.superjob.ru/vakansii/java-develope...,https://www.superjob.ru
8,Back end разработчик на Java,NaN,NaN,https://www.superjob.ru/vakansii/back-end-razr...,https://www.superjob.ru
9,Разработчик Java (Junior),NaN,NaN,https://www.superjob.ru/vakansii/razrabotchik-...,https://www.superjob.ru


In [8]:
client = MongoClient("localhost", 27017)
db = client["Positions"]
superjob = db.superjob

In [9]:
for i in range(len(positions_list)):
    
    if bool(list(superjob.find({"Ссылка на вакансию": positions_list[i][3]}))):
        continue
    
    superjob.insert_one({
        "Название вакансии": positions_list[i][0],
        "Мин. зарплата": positions_list[i][1],
        "Макс. зарплата": positions_list[i][2],
        "Ссылка на вакансию": positions_list[i][3],
        "Сайт": positions_list[i][4]
    })

In [10]:
list(superjob.find({}))

[{'_id': ObjectId('620f9d83bf87d61a15125437'),
  'Название вакансии': 'Java developer (Middle| Senior)',
  'Мин. зарплата': None,
  'Макс. зарплата': 300000,
  'Ссылка на вакансию': 'https://www.superjob.ru/vakansii/java-developer-40917490.html',
  'Сайт': 'https://www.superjob.ru'},
 {'_id': ObjectId('620f9fd1bf87d61a15125438'),
  'Название вакансии': 'Разработчик Java / Программист Java (Junior / Middle Java)',
  'Мин. зарплата': 91000,
  'Макс. зарплата': None,
  'Ссылка на вакансию': 'https://www.superjob.ru/vakansii/razrabotchik-java-41195465.html',
  'Сайт': 'https://www.superjob.ru'},
 {'_id': ObjectId('620fa01b170b0b155d6cea7e'),
  'Название вакансии': 'Java developer',
  'Мин. зарплата': None,
  'Макс. зарплата': 200000,
  'Ссылка на вакансию': 'https://www.superjob.ru/vakansii/java-developer-38286192.html',
  'Сайт': 'https://www.superjob.ru'},
 {'_id': ObjectId('620fa01b170b0b155d6cea7f'),
  'Название вакансии': 'Ведущий разработчик Java',
  'Мин. зарплата': None,
  'Макс. з

In [11]:
min_salary = 200000

In [12]:
for position in superjob.find({}):
    if position["Мин. зарплата"] != None and position["Мин. зарплата"] > min_salary \
                        or position["Макс. зарплата"] != None and position["Макс. зарплата"] > min_salary:
        print(position)

{'_id': ObjectId('620f9d83bf87d61a15125437'), 'Название вакансии': 'Java developer (Middle| Senior)', 'Мин. зарплата': None, 'Макс. зарплата': 300000, 'Ссылка на вакансию': 'https://www.superjob.ru/vakansii/java-developer-40917490.html', 'Сайт': 'https://www.superjob.ru'}
{'_id': ObjectId('620fa01b170b0b155d6cea82'), 'Название вакансии': 'Разработчик Java', 'Мин. зарплата': 250000, 'Макс. зарплата': None, 'Ссылка на вакансию': 'https://www.superjob.ru/vakansii/razrabotchik-java-41542831.html', 'Сайт': 'https://www.superjob.ru'}
{'_id': ObjectId('620fa01b170b0b155d6cea84'), 'Название вакансии': 'Java-developer', 'Мин. зарплата': None, 'Макс. зарплата': 300000, 'Ссылка на вакансию': 'https://www.superjob.ru/vakansii/java-developer-41389819.html', 'Сайт': 'https://www.superjob.ru'}
{'_id': ObjectId('620fa01b170b0b155d6cea9d'), 'Название вакансии': 'Senior Python Backend Developer / Web-разработчик', 'Мин. зарплата': 230000, 'Макс. зарплата': None, 'Ссылка на вакансию': 'https://www.superj